![Verne Academy Summit 2024](https://github.com/javendia/verne-academy-summit-2024/blob/main/header.png?raw=true)

## Paso 1: Ingesta de datos
- Leemos el fichero **CustomerCategories.csv**, especificando el formato (CSV) e indicando que el archivo contiene las cabeceras de las columnas
- Creamos la tabla delta destino en caso de no existir

In [1]:
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import when, lit, col

df = spark.read.load('Files/wwi/CustomerCategories.csv',
    format='csv',
    header=True
)

DeltaTable.createIfNotExists(spark) \
     .tableName("Customer_Categories") \
     .addColumn("CustomerCategoryID", IntegerType()) \
     .addColumn("CustomerCategoryName", StringType()) \
     .addColumn("LastUpdated", TimestampType()) \
     .execute()

deltaTable = DeltaTable.forPath(spark, 'Tables/customer_categories')

StatementMeta(, d64c6625-30c1-44ba-8643-75400042233d, 3, Finished, Available)

## Paso 2: Transformación
- Seleccionamos las columnas necesarias
- Devolvemos un valor controlado para la columna **CustomerCategoryName** en caso de ser nula o vacía

In [2]:
df = df.select("CustomerCategoryID", "CustomerCategoryName") \
        .withColumn("CustomerCategoryName", when((col("CustomerCategoryName").isNull() | (col("CustomerCategoryName")=="")),lit("N/A")).otherwise(col("CustomerCategoryName")))

StatementMeta(, d64c6625-30c1-44ba-8643-75400042233d, 4, Finished, Available)

## Paso 3: Creación de vista temporal
- Creamos la vista temporal **vw_customer_categories** donde almacenar los datos tratados anteriormente

In [3]:
df.createOrReplaceTempView("vw_customer_categories")

StatementMeta(, d64c6625-30c1-44ba-8643-75400042233d, 5, Finished, Available)

## Paso 4: Instrucción MERGE

- Si el valor del registro para la columna **CustomerCategoryID** existe **(MATCHED)** en la tabla destino y alguna columna difiere del registro existente, actualizamos la fila
- Si el valor del registro para la columna **CustomerCategoryID** no existe **(NOT MATCHED)** en la tabla destino, insertamos una nueva fila
- En caso de que el registro de la tabla destino no exista en el fichero origen, eliminamos esa fila

In [4]:
%%sql

MERGE INTO customer_categories AS target
USING vw_customer_categories AS source
ON target.CustomerCategoryID = source.CustomerCategoryID
WHEN MATCHED AND 
(
    target.CustomerCategoryName <> source.CustomerCategoryName
)
THEN
UPDATE SET
    target.CustomerCategoryName = source.CustomerCategoryName
    ,target.LastUpdated = CURRENT_TIMESTAMP()
WHEN NOT MATCHED THEN
INSERT (CustomerCategoryID, CustomerCategoryName, LastUpdated)
VALUES (source.CustomerCategoryID, source.CustomerCategoryName, CURRENT_TIMESTAMP())
WHEN NOT MATCHED BY SOURCE THEN
DELETE;

StatementMeta(, d64c6625-30c1-44ba-8643-75400042233d, 6, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

## Paso 5: Eliminación vista temporal

In [5]:
spark.catalog.dropTempView("vw_customer_categories")

StatementMeta(, d64c6625-30c1-44ba-8643-75400042233d, 7, Finished, Available)

True